# 시간대별 혼잡도와 역세권 건물 상관관계 분석

이 노트북은 지하철 역의 시간대별 혼잡도와 역세권 건물 특성 간의 상관관계를 분석합니다.

## 분석 대상
- **2호선, 4호선, 5호선** 역으로 한정

## 분석 내용
1. 역세권 건물 특성 집계 (용도별 건물 수, 연면적, 세대수 등)
2. 시간대별 혼잡도 데이터 분석
3. 상관관계 분석 및 시각화
4. 건물 용도와 시간대별 혼잡도 패턴 분석

In [ ]:
from huggingface_hub import hf_hub_download
import sqlite3
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# 설정
DB_PATH = "db/subway.db"     # 리포지토리 내 파일 경로


def get_connection():
    return sqlite3.connect(DB_PATH)

print("라이브러리 로드 완료")

c:\USERS\ADMINISTRATOR\DOCUMENTS\SUBWAY-CONGESTION-ANALYSIS\.VENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


라이브러리 로드 완료


## 1. 데이터 로드

In [2]:
conn = get_connection()

# 분석 대상 호선 설정
TARGET_LINES = ['2호선', '4호선', '5호선']

# 역 정보 로드 (2, 4, 5호선만)
stations_query = """
SELECT 
    s.station_id,
    s.station_name_kr,
    sr.station_code,
    sr.admin_dong_code,
    sr.admin_dong_name,
    l.line_name
FROM Stations s
JOIN Station_Routes sr ON s.station_id = sr.station_id
JOIN Lines l ON sr.line_id = l.line_id
WHERE l.line_name IN ('2호선', '4호선', '5호선')
"""
df_stations = pd.read_sql(stations_query, conn)

# 대상 역 ID 목록
target_station_ids = df_stations['station_id'].unique().tolist()
target_station_codes = df_stations['station_code'].unique().tolist()

# 역세권 건물 정보 로드 (대상 역만)
buildings_query = f"""
SELECT 
    b.*,
    s.station_name_kr
FROM Station_Catchment_Buildings b
JOIN Stations s ON b.station_id = s.station_id
WHERE b.station_id IN ({','.join(map(str, target_station_ids))})
"""
df_buildings = pd.read_sql(buildings_query, conn)

# 혼잡도 데이터 로드 (대상 역만)
congestion_query = f"""
SELECT * FROM Station_Congestion
WHERE station_code IN ({','.join([f"'{c}'" for c in target_station_codes])})
"""
df_congestion = pd.read_sql(congestion_query, conn)

print(f"분석 대상 호선: {TARGET_LINES}")
print(f"역 정보: {len(df_stations):,} rows ({df_stations['station_id'].nunique()}개 역)")
print(f"역세권 건물: {len(df_buildings):,} rows")
print(f"혼잡도 데이터: {len(df_congestion):,} rows")

# 호선별 역 수
print(f"\n호선별 역 수:")
print(df_stations.groupby('line_name')['station_id'].nunique())

display(df_congestion.head())

분석 대상 호선: ['2호선', '4호선', '5호선']
역 정보: 132 rows (125개 역)
역세권 건물: 239,295 rows
혼잡도 데이터: 130,960 rows

호선별 역 수:
line_name
2호선    50
4호선    26
5호선    56
Name: station_id, dtype: int64


,congestion_id,quarter_code,station_code,day_of_week,is_upline,time_slot,congestion_level
0,39,20234,201,0,0,0,20.5
1,1635,20234,201,0,0,1,27.3
2,3270,20234,201,0,0,2,27.7
3,4905,20234,201,0,0,3,41.4
4,6540,20234,201,0,0,4,67.2


### 1.1 혼잡도 데이터 기본 정보

In [3]:
print("=== 혼잡도 데이터 기본 정보 ===")
print(f"분기 코드: {df_congestion['quarter_code'].unique()}")
print(f"요일 구분: 0=평일, 1=토요일, 2=일요일")
print(f"시간대 범위: {df_congestion['time_slot'].min()} ~ {df_congestion['time_slot'].max()}")
print(f"혼잡도 범위: {df_congestion['congestion_level'].min():.1f} ~ {df_congestion['congestion_level'].max():.1f}")
print(f"\n역 코드 수: {df_congestion['station_code'].nunique()}")

=== 혼잡도 데이터 기본 정보 ===
분기 코드: ['20234' '20241' '20242' '20244' '20251']
요일 구분: 0=평일, 1=토요일, 2=일요일
시간대 범위: 0 ~ 38
혼잡도 범위: 0.0 ~ 192.8

역 코드 수: 132


In [4]:
# 시간대 슬롯을 실제 시간으로 변환 (05:30 = 0, 06:00 = 1, ...)
def slot_to_time(slot):
    """시간대 슬롯을 실제 시간 문자열로 변환"""
    base_hour = 5
    base_minute = 30
    total_minutes = base_hour * 60 + base_minute + slot * 30
    hour = (total_minutes // 60) % 24
    minute = total_minutes % 60
    return f"{hour:02d}:{minute:02d}"

def slot_to_hour(slot):
    """시간대 슬롯을 시간(hour)으로 변환"""
    base_hour = 5
    base_minute = 30
    total_minutes = base_hour * 60 + base_minute + slot * 30
    return (total_minutes // 60) % 24

# 시간대 그룹 분류
def categorize_time_period(slot):
    """시간대를 출근/점심/퇴근/야간 등으로 분류"""
    hour = slot_to_hour(slot)
    if 0 <= hour < 6:
        return '심야(00-06)'
    elif 6 <= hour < 9:
        return '출근(06-09)'
    elif 9 <= hour < 12:
        return '오전(09-12)'
    elif 12 <= hour < 14:
        return '점심(12-14)'
    elif 14 <= hour < 18:
        return '오후(14-18)'
    elif 18 <= hour < 21:
        return '퇴근(18-21)'
    else:
        return '야간(21-24)'

df_congestion['time_str'] = df_congestion['time_slot'].apply(slot_to_time)
df_congestion['hour'] = df_congestion['time_slot'].apply(slot_to_hour)
df_congestion['time_period'] = df_congestion['time_slot'].apply(categorize_time_period)

print("시간대 슬롯 매핑 예시:")
for slot in [0, 1, 2, 5, 10, 15, 20, 25, 30, 35, 38]:
    print(f"  slot {slot:2d} -> {slot_to_time(slot)} ({categorize_time_period(slot)})")

시간대 슬롯 매핑 예시:
  slot  0 -> 05:30 (심야(00-06))
  slot  1 -> 06:00 (출근(06-09))
  slot  2 -> 06:30 (출근(06-09))
  slot  5 -> 08:00 (출근(06-09))
  slot 10 -> 10:30 (오전(09-12))
  slot 15 -> 13:00 (점심(12-14))
  slot 20 -> 15:30 (오후(14-18))
  slot 25 -> 18:00 (퇴근(18-21))
  slot 30 -> 20:30 (퇴근(18-21))
  slot 35 -> 23:00 (야간(21-24))
  slot 38 -> 00:30 (심야(00-06))


## 2. 역별 건물 특성 집계

In [5]:
# 용도 카테고리 분류
def categorize_usage(usage_type):
    if pd.isna(usage_type):
        return '기타'
    usage_type = str(usage_type)
    if '주택' in usage_type or '공동주택' in usage_type:
        return '주거'
    elif '근린생활' in usage_type or '판매' in usage_type:
        return '상업'
    elif '업무' in usage_type:
        return '업무'
    elif '교육' in usage_type or '학교' in usage_type:
        return '교육'
    elif '숙박' in usage_type:
        return '숙박'
    elif '공장' in usage_type or '창고' in usage_type:
        return '산업'
    elif '종교' in usage_type or '문화' in usage_type:
        return '문화종교'
    else:
        return '기타'

df_buildings['usage_category'] = df_buildings['usage_type'].apply(categorize_usage)

# 역별 건물 특성 집계
station_building_stats = df_buildings.groupby('station_id').agg({
    'id': 'count',
    'height': 'mean',
    'floor_area': 'sum',
    'households': 'sum',
    'families': 'sum'
}).rename(columns={
    'id': 'building_count',
    'height': 'avg_height',
    'floor_area': 'total_floor_area',
    'households': 'total_households',
    'families': 'total_families'
}).reset_index()

# 용도별 건물 수 피벗
usage_pivot = df_buildings.groupby(['station_id', 'usage_category']).size().unstack(fill_value=0)
usage_pivot.columns = [f'cnt_{col}' for col in usage_pivot.columns]
usage_pivot = usage_pivot.reset_index()

# 합치기
station_building_stats = station_building_stats.merge(usage_pivot, on='station_id', how='left')

# 역명 추가
station_names = df_buildings[['station_id', 'station_name_kr']].drop_duplicates()
station_building_stats = station_building_stats.merge(station_names, on='station_id', how='left')

print(f"역별 건물 특성 집계: {len(station_building_stats)} 역")
display(station_building_stats.head())

역별 건물 특성 집계: 119 역


,station_id,building_count,avg_height,total_floor_area,total_households,total_families,cnt_교육,cnt_기타,cnt_문화종교,cnt_산업,cnt_상업,cnt_숙박,cnt_업무,cnt_주거,station_name_kr
0,1,2642,4.852903,170912.290040,6086.0,1138.0,10,906,20,30,724,20,106,826,서울
1,2,1588,7.841936,184587.381544,5820.0,1024.0,40,516,76,6,548,58,276,68,시청
2,4,6852,2.827830,357611.362872,13998.0,1818.0,36,1374,102,57,3729,219,111,1224,종로3가
3,6,5968,2.926198,311511.743666,11734.0,2276.0,10,1504,18,128,1966,56,52,2234,동대문
4,7,5238,3.464670,269629.740356,10138.0,1890.0,74,1698,16,66,1392,40,158,1794,신설동


## 3. 역별 시간대별 혼잡도 집계

In [6]:
# station_code를 station_id로 매핑
station_code_to_id = df_stations[['station_id', 'station_code']].drop_duplicates()

# 혼잡도 데이터에 station_id 추가
df_congestion = df_congestion.merge(station_code_to_id, on='station_code', how='left')

print(f"혼잡도 데이터 중 station_id 매핑된 수: {df_congestion['station_id'].notna().sum():,}")

혼잡도 데이터 중 station_id 매핑된 수: 130,960


In [7]:
# 평일 데이터만 사용 (day_of_week = 0)
df_congestion_weekday = df_congestion[df_congestion['day_of_week'] == 0].copy()

# 역별, 시간대별 평균 혼잡도 (상행/하행 평균)
congestion_by_time = df_congestion_weekday.groupby(['station_id', 'time_period']).agg({
    'congestion_level': 'mean'
}).reset_index()

# 피벗
congestion_pivot = congestion_by_time.pivot(
    index='station_id', 
    columns='time_period', 
    values='congestion_level'
).reset_index()

# 컬럼명 정리
congestion_pivot.columns.name = None

# 역별 전체 평균 혼잡도도 계산
avg_congestion = df_congestion_weekday.groupby('station_id')['congestion_level'].mean().reset_index()
avg_congestion.columns = ['station_id', 'avg_congestion']

congestion_pivot = congestion_pivot.merge(avg_congestion, on='station_id', how='left')

print(f"혼잡도 피벗 데이터: {len(congestion_pivot)} 역")
display(congestion_pivot.head())

혼잡도 피벗 데이터: 125 역


,station_id,심야(00-06),야간(21-24),오전(09-12),오후(14-18),점심(12-14),출근(06-09),퇴근(18-21),avg_congestion
0,1,10.696667,23.130000,29.615000,38.28375,31.3400,28.188333,34.680000,29.676923
1,2,11.956667,40.073333,34.621667,48.51375,38.4075,43.441667,56.310000,41.648462
2,4,9.046667,25.978333,26.166667,39.68875,26.7500,42.086667,42.946667,32.685128
3,6,21.623333,42.155000,47.408333,62.21125,46.4325,57.651667,59.538333,50.995128
4,7,1.816667,1.986667,8.110000,6.90625,6.0800,13.355000,4.513333,6.482308


## 4. 데이터 병합 및 상관관계 분석

In [8]:
# 건물 특성과 혼잡도 데이터 병합
df_analysis = station_building_stats.merge(congestion_pivot, on='station_id', how='inner')

print(f"분석 대상 역 수: {len(df_analysis)}")
display(df_analysis.head())

분석 대상 역 수: 119


,station_id,building_count,avg_height,total_floor_area,total_households,total_families,cnt_교육,cnt_기타,cnt_문화종교,cnt_산업,...,cnt_주거,station_name_kr,심야(00-06),야간(21-24),오전(09-12),오후(14-18),점심(12-14),출근(06-09),퇴근(18-21),avg_congestion
0,1,2642,4.852903,170912.290040,6086.0,1138.0,10,906,20,30,...,826,서울,10.696667,23.130000,29.615000,38.28375,31.3400,28.188333,34.680000,29.676923
1,2,1588,7.841936,184587.381544,5820.0,1024.0,40,516,76,6,...,68,시청,11.956667,40.073333,34.621667,48.51375,38.4075,43.441667,56.310000,41.648462
2,4,6852,2.827830,357611.362872,13998.0,1818.0,36,1374,102,57,...,1224,종로3가,9.046667,25.978333,26.166667,39.68875,26.7500,42.086667,42.946667,32.685128
3,6,5968,2.926198,311511.743666,11734.0,2276.0,10,1504,18,128,...,2234,동대문,21.623333,42.155000,47.408333,62.21125,46.4325,57.651667,59.538333,50.995128
4,7,5238,3.464670,269629.740356,10138.0,1890.0,74,1698,16,66,...,1794,신설동,1.816667,1.986667,8.110000,6.90625,6.0800,13.355000,4.513333,6.482308


In [9]:
# 상관관계 분석을 위한 변수 선택
building_features = ['building_count', 'avg_height', 'total_floor_area', 'total_households']
usage_features = [c for c in df_analysis.columns if c.startswith('cnt_') and c != 'cnt_기타']
building_features.extend(usage_features)

time_features = ['심야(00-06)', '출근(06-09)', '오전(09-12)', '점심(12-14)', 
                 '오후(14-18)', '퇴근(18-21)', '야간(21-24)', 'avg_congestion']
time_features = [f for f in time_features if f in df_analysis.columns]

# 상관계수 계산
all_features = building_features + time_features
corr_matrix = df_analysis[all_features].corr()

# 건물 특성과 혼잡도 간의 상관관계만 추출
corr_building_congestion = corr_matrix.loc[building_features, time_features]

print("건물 특성과 시간대별 혼잡도 간 상관계수:")
display(corr_building_congestion.round(3))

건물 특성과 시간대별 혼잡도 간 상관계수:


,심야(00-06),출근(06-09),오전(09-12),점심(12-14),오후(14-18),퇴근(18-21),야간(21-24),avg_congestion
building_count,0.113,0.159,0.143,0.181,0.156,0.106,0.147,0.149
avg_height,0.174,0.129,0.171,0.173,0.208,0.232,0.238,0.202
total_floor_area,0.320,0.310,0.290,0.306,0.278,0.241,0.298,0.295
total_households,0.253,0.280,0.250,0.277,0.249,0.211,0.267,0.262
cnt_교육,0.072,0.071,0.135,0.179,0.184,0.142,0.181,0.150
cnt_문화종교,0.016,0.072,0.112,0.168,0.169,0.133,0.148,0.134
cnt_산업,-0.081,-0.051,0.033,0.104,0.088,0.066,0.080,0.048
cnt_상업,0.076,0.124,0.157,0.220,0.206,0.155,0.203,0.178
cnt_숙박,0.069,0.098,0.143,0.197,0.194,0.154,0.188,0.164
cnt_업무,0.249,0.189,0.324,0.371,0.390,0.367,0.419,0.352


In [10]:
# 상관관계 히트맵
fig = go.Figure(data=go.Heatmap(
    z=corr_building_congestion.values,
    x=corr_building_congestion.columns,
    y=corr_building_congestion.index,
    colorscale='RdBu_r',
    zmid=0,
    text=corr_building_congestion.values.round(2),
    texttemplate='%{text}',
    textfont={'size': 9},
    hoverongaps=False,
    colorbar=dict(title='상관계수')
))

fig.update_layout(
    title='역세권 건물 특성과 시간대별 혼잡도 상관관계',
    xaxis_title='시간대별 혼잡도',
    yaxis_title='건물 특성',
    height=500,
    width=800
)

fig.show()

## 5. 주요 상관관계 상세 분석

In [11]:
# 주거 건물과 시간대별 혼잡도 관계
if 'cnt_주거' in df_analysis.columns:
    time_cols = [c for c in time_features if c != 'avg_congestion' and c in df_analysis.columns]
    n_cols = len(time_cols)
    
    fig = make_subplots(rows=2, cols=4, subplot_titles=time_cols[:8])
    
    positions = [(1,1), (1,2), (1,3), (1,4), (2,1), (2,2), (2,3), (2,4)]

    for i, col in enumerate(time_cols[:8]):
        if col in df_analysis.columns:
            corr, _ = stats.pearsonr(
                df_analysis['cnt_주거'].fillna(0), 
                df_analysis[col].fillna(0)
            )
            
            pos = positions[i]
            fig.add_trace(
                go.Scatter(
                    x=df_analysis['cnt_주거'],
                    y=df_analysis[col],
                    mode='markers',
                    marker=dict(size=5, opacity=0.6),
                    text=df_analysis['station_name_kr'],
                    name=f'r={corr:.2f}',
                    showlegend=False
                ),
                row=pos[0], col=pos[1]
            )
            
            # 추세선 추가
            x_valid = df_analysis['cnt_주거'].fillna(0)
            y_valid = df_analysis[col].fillna(0)
            z = np.polyfit(x_valid, y_valid, 1)
            p = np.poly1d(z)
            x_line = np.linspace(x_valid.min(), x_valid.max(), 100)
            fig.add_trace(
                go.Scatter(x=x_line, y=p(x_line), mode='lines', 
                          line=dict(color='red', dash='dash'),
                          showlegend=False),
                row=pos[0], col=pos[1]
            )

    fig.update_layout(
        height=500, 
        width=1000,
        title_text='주거 건물 수와 시간대별 혼잡도 관계'
    )
    fig.update_xaxes(title_text='주거 건물 수')
    fig.update_yaxes(title_text='혼잡도')
    fig.show()

In [12]:
# 상업 건물과 시간대별 혼잡도 관계
if 'cnt_상업' in df_analysis.columns:
    time_cols = [c for c in time_features if c != 'avg_congestion' and c in df_analysis.columns]
    
    fig = make_subplots(rows=2, cols=4, subplot_titles=time_cols[:8])
    positions = [(1,1), (1,2), (1,3), (1,4), (2,1), (2,2), (2,3), (2,4)]

    for i, col in enumerate(time_cols[:8]):
        if col in df_analysis.columns:
            corr, _ = stats.pearsonr(
                df_analysis['cnt_상업'].fillna(0), 
                df_analysis[col].fillna(0)
            )
            
            pos = positions[i]
            fig.add_trace(
                go.Scatter(
                    x=df_analysis['cnt_상업'],
                    y=df_analysis[col],
                    mode='markers',
                    marker=dict(size=5, opacity=0.6, color='orange'),
                    text=df_analysis['station_name_kr'],
                    showlegend=False
                ),
                row=pos[0], col=pos[1]
            )

    fig.update_layout(
        height=500, 
        width=1000,
        title_text='상업 건물 수와 시간대별 혼잡도 관계'
    )
    fig.update_xaxes(title_text='상업 건물 수')
    fig.update_yaxes(title_text='혼잡도')
    fig.show()

In [13]:
# 업무 건물과 시간대별 혼잡도 관계
if 'cnt_업무' in df_analysis.columns:
    time_cols = [c for c in time_features if c != 'avg_congestion' and c in df_analysis.columns]
    
    fig = make_subplots(rows=2, cols=4, subplot_titles=time_cols[:8])
    positions = [(1,1), (1,2), (1,3), (1,4), (2,1), (2,2), (2,3), (2,4)]

    for i, col in enumerate(time_cols[:8]):
        if col in df_analysis.columns:
            corr, _ = stats.pearsonr(
                df_analysis['cnt_업무'].fillna(0), 
                df_analysis[col].fillna(0)
            )
            
            pos = positions[i]
            fig.add_trace(
                go.Scatter(
                    x=df_analysis['cnt_업무'],
                    y=df_analysis[col],
                    mode='markers',
                    marker=dict(size=5, opacity=0.6, color='green'),
                    text=df_analysis['station_name_kr'],
                    showlegend=False
                ),
                row=pos[0], col=pos[1]
            )

    fig.update_layout(
        height=500, 
        width=1000,
        title_text='업무 건물 수와 시간대별 혼잡도 관계'
    )
    fig.update_xaxes(title_text='업무 건물 수')
    fig.update_yaxes(title_text='혼잡도')
    fig.show()

## 6. 시간대별 혼잡도 패턴 분석

In [14]:
# 전체 역의 시간대별 평균 혼잡도 패턴
time_pattern = df_congestion_weekday.groupby('time_slot')['congestion_level'].mean().reset_index()
time_pattern['time_str'] = time_pattern['time_slot'].apply(slot_to_time)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=time_pattern['time_str'],
    y=time_pattern['congestion_level'],
    mode='lines+markers',
    line=dict(color='blue', width=2),
    marker=dict(size=6)
))

fig.update_layout(
    title='평일 시간대별 평균 혼잡도',
    xaxis_title='시간',
    yaxis_title='평균 혼잡도',
    height=400,
    xaxis=dict(tickangle=45)
)
fig.show()

In [15]:
# 건물 용도 비율 계산
usage_cols = [c for c in df_analysis.columns if c.startswith('cnt_')]
for col in usage_cols:
    ratio_col = col.replace('cnt_', 'ratio_')
    df_analysis[ratio_col] = df_analysis[col] / df_analysis['building_count']

# 주거 비율 상위/하위 역 비교
df_analysis['주거비율_그룹'] = pd.qcut(df_analysis['ratio_주거'].fillna(0), q=3, labels=['하위', '중위', '상위'])

# 그룹별 시간대 혼잡도 패턴
group_patterns = df_analysis.groupby('주거비율_그룹')[time_features].mean()

fig = go.Figure()
colors = {'하위': 'red', '중위': 'gray', '상위': 'blue'}

for group in ['하위', '중위', '상위']:
    if group in group_patterns.index:
        values = group_patterns.loc[group, [c for c in time_features if c != 'avg_congestion']]
        fig.add_trace(go.Bar(
            name=f'주거 비율 {group}',
            x=values.index,
            y=values.values,
            marker_color=colors[group]
        ))

fig.update_layout(
    title='주거 건물 비율별 시간대 혼잡도 비교',
    xaxis_title='시간대',
    yaxis_title='평균 혼잡도',
    barmode='group',
    height=400
)
fig.show()

In [16]:
# 업무 비율 상위/하위 역 비교
if 'ratio_업무' in df_analysis.columns:
    df_analysis['업무비율_그룹'] = pd.qcut(
        df_analysis['ratio_업무'].fillna(0).clip(lower=0.001), 
        q=3, 
        labels=['하위', '중위', '상위'],
        duplicates='drop'
    )

    group_patterns = df_analysis.groupby('업무비율_그룹')[time_features].mean()

    fig = go.Figure()
    colors = {'하위': 'lightgreen', '중위': 'gray', '상위': 'darkgreen'}

    for group in ['하위', '중위', '상위']:
        if group in group_patterns.index:
            values = group_patterns.loc[group, [c for c in time_features if c != 'avg_congestion']]
            fig.add_trace(go.Bar(
                name=f'업무 비율 {group}',
                x=values.index,
                y=values.values,
                marker_color=colors[group]
            ))

    fig.update_layout(
        title='업무 건물 비율별 시간대 혼잡도 비교',
        xaxis_title='시간대',
        yaxis_title='평균 혼잡도',
        barmode='group',
        height=400
    )
    fig.show()

## 7. 역 클러스터링 (건물 특성 기반)

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# 클러스터링에 사용할 피처
cluster_features = ['building_count', 'avg_height', 'total_floor_area']
ratio_features = [c for c in df_analysis.columns if c.startswith('ratio_') and c not in ['ratio_기타']]
ratio_features = [f for f in ratio_features if f in df_analysis.columns and not f.endswith('_그룹')]

cluster_features.extend(ratio_features[:5])  # 상위 5개 용도만 사용
cluster_features = [f for f in cluster_features if f in df_analysis.columns]

print(f"클러스터링 피처: {cluster_features}")

# 결측치 처리 및 스케일링
X = df_analysis[cluster_features].fillna(0)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# K-means 클러스터링
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_analysis['cluster'] = kmeans.fit_predict(X_scaled)

print("\n클러스터별 역 수:")
print(df_analysis['cluster'].value_counts().sort_index())

클러스터링 피처: ['building_count', 'avg_height', 'total_floor_area', 'ratio_교육', 'ratio_문화종교', 'ratio_산업', 'ratio_상업', 'ratio_숙박']

클러스터별 역 수:
cluster
0    28
1    23
2     4
3    64
Name: count, dtype: int64


In [18]:
# 클러스터별 시간대 혼잡도 패턴
time_cols_for_viz = [c for c in time_features if c != 'avg_congestion' and c in df_analysis.columns]
cluster_patterns = df_analysis.groupby('cluster')[time_cols_for_viz].mean()

fig = go.Figure()
colors = ['blue', 'red', 'green', 'purple']

for cluster in sorted(cluster_patterns.index):
    fig.add_trace(go.Scatter(
        x=time_cols_for_viz,
        y=cluster_patterns.loc[cluster].values,
        mode='lines+markers',
        name=f'클러스터 {cluster}',
        line=dict(color=colors[cluster % len(colors)], width=2)
    ))

fig.update_layout(
    title='클러스터별 시간대 혼잡도 패턴',
    xaxis_title='시간대',
    yaxis_title='평균 혼잡도',
    height=450
)
fig.show()

In [19]:
# 클러스터별 건물 특성 요약
summary_cols = ['building_count', 'avg_height', 'total_floor_area', 'total_households', 'avg_congestion']
usage_cols_summary = [c for c in df_analysis.columns if c.startswith('cnt_') and c != 'cnt_기타']
summary_cols.extend(usage_cols_summary)
summary_cols = [c for c in summary_cols if c in df_analysis.columns]

cluster_summary = df_analysis.groupby('cluster')[summary_cols].mean().round(1)

print("클러스터별 특성 평균:")
display(cluster_summary.T)

클러스터별 특성 평균:


cluster,0,1,2,3
building_count,4124.8,796.8,1189.2,1573.7
avg_height,6.3,10.7,6.9,6.2
total_floor_area,330329.3,84048.3,72976.0,120223.0
total_households,10480.4,3086.2,3146.5,4046.6
avg_congestion,39.2,34.5,33.4,31.9
cnt_교육,30.4,25.5,6.8,12.5
cnt_문화종교,18.2,12.3,2.2,6.5
cnt_산업,40.2,3.0,124.8,4.8
cnt_상업,1289.5,233.9,298.8,271.9
cnt_숙박,44.2,12.1,1.2,3.4


In [20]:
# 각 클러스터 대표 역
print("클러스터별 대표 역:")
for cluster in sorted(df_analysis['cluster'].unique()):
    cluster_data = df_analysis[df_analysis['cluster'] == cluster]
    stations = cluster_data.nlargest(5, 'avg_congestion')['station_name_kr'].tolist()
    avg_cong = cluster_data['avg_congestion'].mean()
    print(f"\n클러스터 {cluster} (평균 혼잡도: {avg_cong:.1f}):")
    print(f"  {', '.join(stations)}")

클러스터별 대표 역:

클러스터 0 (평균 혼잡도: 39.2):
  교대, 동대문, 역삼, 서울대입구, 신당

클러스터 1 (평균 혼잡도: 34.5):
  강남, 혜화, 광나루, 선릉, 시청

클러스터 2 (평균 혼잡도: 33.4):
  신도림, 뚝섬, 성수, 도림천

클러스터 3 (평균 혼잡도: 31.9):
  방배, 서초, 길동, 낙성대, 남태령


## 8. 출퇴근 시간대 혼잡도 차이 분석

In [21]:
# 출근 vs 퇴근 혼잡도 차이
if '출근(06-09)' in df_analysis.columns and '퇴근(18-21)' in df_analysis.columns:
    df_analysis['출퇴근_차이'] = df_analysis['퇴근(18-21)'] - df_analysis['출근(06-09)']
    
    # 출근 혼잡도가 높은 역 (주거 지역 추정)
    morning_heavy = df_analysis.nlargest(10, '출근(06-09)')[['station_name_kr', '출근(06-09)', '퇴근(18-21)', 'cnt_주거', 'cnt_업무']]
    
    # 퇴근 혼잡도가 높은 역 (업무 지역 추정)
    evening_heavy = df_analysis.nlargest(10, '퇴근(18-21)')[['station_name_kr', '출근(06-09)', '퇴근(18-21)', 'cnt_주거', 'cnt_업무']]
    
    print("=== 출근 시간대 혼잡도 Top 10 (주거 지역 추정) ===")
    display(morning_heavy)
    
    print("\n=== 퇴근 시간대 혼잡도 Top 10 (업무 지역 추정) ===")
    display(evening_heavy)

=== 출근 시간대 혼잡도 Top 10 (주거 지역 추정) ===


,station_name_kr,출근(06-09),퇴근(18-21),cnt_주거,cnt_업무
28,방배,63.790000,67.921667,561,35
104,아차산,63.663333,56.481667,1713,12
75,남태령,62.971667,62.635000,182,0
30,낙성대,62.705000,60.038333,1179,53
27,서초,61.378333,68.243333,199,125
105,광나루,61.321667,55.633333,320,3
38,문래,60.776667,57.318333,166,41
37,신도림,59.665000,47.013333,279,17
31,서울대입구,59.546667,55.533333,1491,126
103,군자,59.430000,52.656667,4032,64



=== 퇴근 시간대 혼잡도 Top 10 (업무 지역 추정) ===


,station_name_kr,출근(06-09),퇴근(18-21),cnt_주거,cnt_업무
26,교대,59.008333,68.596667,520,290
25,강남,51.328333,68.450000,110,118
27,서초,61.378333,68.243333,199,125
24,역삼,46.266667,67.933333,433,148
28,방배,63.790000,67.921667,561,35
108,길동,58.531667,65.015000,933,37
75,남태령,62.971667,62.635000,182,0
30,낙성대,62.705000,60.038333,1179,53
109,굽은다리,55.815000,59.708333,1361,6
3,동대문,57.651667,59.538333,2234,52


In [22]:
# 출근 vs 퇴근 혼잡도 산점도
if '출근(06-09)' in df_analysis.columns and '퇴근(18-21)' in df_analysis.columns:
    fig = go.Figure()
    
    # 주거 비율에 따른 색상
    fig.add_trace(go.Scatter(
        x=df_analysis['출근(06-09)'],
        y=df_analysis['퇴근(18-21)'],
        mode='markers',
        marker=dict(
            size=8,
            color=df_analysis['ratio_주거'].fillna(0),
            colorscale='RdYlBu_r',
            showscale=True,
            colorbar=dict(title='주거비율')
        ),
        text=df_analysis['station_name_kr'],
        hovertemplate='%{text}<br>출근: %{x:.1f}<br>퇴근: %{y:.1f}<extra></extra>'
    ))
    
    # 대각선 (출근=퇴근)
    max_val = max(df_analysis['출근(06-09)'].max(), df_analysis['퇴근(18-21)'].max())
    fig.add_trace(go.Scatter(
        x=[0, max_val],
        y=[0, max_val],
        mode='lines',
        line=dict(color='gray', dash='dash'),
        showlegend=False
    ))
    
    fig.update_layout(
        title='출근 vs 퇴근 시간대 혼잡도 (색상: 주거 비율)',
        xaxis_title='출근 시간대(06-09) 혼잡도',
        yaxis_title='퇴근 시간대(18-21) 혼잡도',
        height=500,
        width=600
    )
    fig.show()
    
    print("\n* 대각선 위: 퇴근 혼잡도 > 출근 혼잡도 (업무/상업 지역)")
    print("* 대각선 아래: 출근 혼잡도 > 퇴근 혼잡도 (주거 지역)")


* 대각선 위: 퇴근 혼잡도 > 출근 혼잡도 (업무/상업 지역)
* 대각선 아래: 출근 혼잡도 > 퇴근 혼잡도 (주거 지역)


## 9. 분석 결과 요약

In [ ]:
print("=" * 70)
print("시간대별 혼잡도와 역세권 건물 상관관계 분석 결과 요약")
print("=" * 70)

print(f"\n[분석 개요]")
print(f"  - 분석 대상 역: {len(df_analysis)}개")
print(f"  - 총 건물 수: {df_buildings['id'].count():,}개")
print(f"  - 혼잡도 데이터: {len(df_congestion):,}건")

print(f"\n[주요 상관관계 (|r| > 0.1)]")
for building_col in building_features:d
    if building_col in df_analysis.columns:
        significant_corrs = []
        for time_col in time_features:
            if time_col in df_analysis.columns:
                corr = df_analysis[building_col].corr(df_analysis[time_col])
                if abs(corr) > 0.1:
                    significant_corrs.append((time_col, corr))
        if significant_corrs:
            print(f"  {building_col}:")
            for time_col, corr in sorted(significant_corrs, key=lambda x: abs(x[1]), reverse=True)[:3]:
                direction = "양" if corr > 0 else "음"
                print(f"    - {time_col}: r={corr:.3f} ({direction}의 상관)")

print(f"\n[클러스터 특성]")
for cluster in sorted(df_analysis['cluster'].unique()):
    cluster_data = df_analysis[df_analysis['cluster'] == cluster]
    n_stations = len(cluster_data)
    avg_cong = cluster_data['avg_congestion'].mean()
    
    # 주요 건물 용도
    usage_cols = [c for c in cluster_data.columns if c.startswith('cnt_') and c != 'cnt_기타']
    if usage_cols:
        main_usage = max(usage_cols, key=lambda x: cluster_data[x].mean())
        main_usage_name = main_usage.replace('cnt_', '')
    else:
        main_usage_name = 'N/A'
    
    print(f"  클러스터 {cluster}: {n_stations}개 역, 평균 혼잡도 {avg_cong:.1f}, 주요 용도: {main_usage_name}")

conn.close()
print("\n" + "=" * 70)
print("분석 완료!")

시간대별 혼잡도와 역세권 건물 상관관계 분석 결과 요약

[분석 개요]
  - 분석 대상 역: 119개
  - 총 건물 수: 239,295개
  - 혼잡도 데이터: 130,960건

[주요 상관관계 (|r| > 0.1)]
  building_count:
    - 점심(12-14): r=0.181 (양의 상관)
    - 출근(06-09): r=0.159 (양의 상관)
    - 오후(14-18): r=0.156 (양의 상관)
  avg_height:
    - 야간(21-24): r=0.238 (양의 상관)
    - 퇴근(18-21): r=0.232 (양의 상관)
    - 오후(14-18): r=0.208 (양의 상관)
  total_floor_area:
    - 심야(00-06): r=0.320 (양의 상관)
    - 출근(06-09): r=0.310 (양의 상관)
    - 점심(12-14): r=0.306 (양의 상관)
  total_households:
    - 출근(06-09): r=0.280 (양의 상관)
    - 점심(12-14): r=0.277 (양의 상관)
    - 야간(21-24): r=0.267 (양의 상관)
  cnt_교육:
    - 오후(14-18): r=0.184 (양의 상관)
    - 야간(21-24): r=0.181 (양의 상관)
    - 점심(12-14): r=0.179 (양의 상관)
  cnt_문화종교:
    - 오후(14-18): r=0.169 (양의 상관)
    - 점심(12-14): r=0.168 (양의 상관)
    - 야간(21-24): r=0.148 (양의 상관)
  cnt_산업:
    - 점심(12-14): r=0.104 (양의 상관)
  cnt_상업:
    - 점심(12-14): r=0.220 (양의 상관)
    - 오후(14-18): r=0.206 (양의 상관)
    - 야간(21-24): r=0.203 (양의 상관)
  cnt_숙박:
    - 점심(12-14): r=0.197 (